In [1]:
# 本文说明设计回测系统时，如何实现行情数据处理部分，包括接口基类、历史和实时tick 示例。

from __future__ import print_function
from abc import ABCMeta, abstractmethod 
import datetime
import threading  
import os, os.path
import numpy as np 
import pandas as pd
import akshare as ak
#from event import MarketEvent

##### 1 策略基类

In [3]:
# 策略基类
class Strategy(object): 
    """
    在Python中，__metaclass__ 是一个特殊的属性，用于在创建类时改变类的行为。这个属性通常在类的定义中设置，
    以指示应该使用哪个元类（metaclass）来创建该类。

    ABCMeta 是Python的 abc 模块中定义的一个元类。它用于创建抽象基类（Abstract Base Classes，简称ABCs）。
    抽象基类允许你定义接口，也就是说，你可以定义一些方法，但不实现它们。然后，其他类可以继承这个抽象基类，并必须实现所有的抽象方法。
    使用 ABCMeta 作为元类可以确保子类实现所有的抽象方法。如果子类没有实现所有的抽象方法，那么在尝试创建该子类的实例时，
    Python会引发一个 TypeError。
    """
    __metaclass__ = ABCMeta
    
@abstractmethod
def calculate_signals(self):
    """
    触发信号的逻辑
    """
    raise NotImplementedError("Should implement calculate_signals()")


In [4]:

def create_sharpe_ratio(returns, periods=252): 
    """
    计算sharp
    Parameters:
    returns - A pandas Series representing period percentage returns. 
    periods - Daily (252), Hourly (252*4), Minutely(252*4*60) etc. 
    """
    return np.sqrt(periods) * (np.mean(returns)) / np.std(returns)

def create_drawdowns(pnl): 
    """
    Calculate the largest peak-to-trough drawdown of the PnL curve as well as the duration of the drawdown. 
    Requires that the pnl_returns is a pandas Series.
    Parameters:
    pnl - A pandas Series representing period percentage returns.
    Returns:
    drawdown, duration - Highest peak-to-trough drawdown and duration. """
    # Calculate the cumulative returns curve
    # and set up the High Water Mark
    hwm = [0]
    # Create the drawdown and duration series
    idx = pnl.index
    drawdown = pd.Series(index = idx)
    duration = pd.Series(index = idx)
    # Loop over the index range
    for t in range(1, len(idx)):
        # hwm 总是保存到当前周期位置的最大收益率
        hwm.append(max(hwm[t-1], pnl[t]))
        # 计算 历史最大收益率 - 当前收益率
        drawdown[t]= (hwm[t]-pnl[t])
        # 最大回撤周期
        duration[t]= (0 if drawdown[t] == 0 else duration[t-1]+1)
    return drawdown, drawdown.max(), duration.max()

#####  从akshare 获取历史数据

In [6]:
# 定于数据处理的抽象基类
class DataHandler(object): 
    # ABCMeta 是 Python 的一个内置元类（metaclass），用于实现抽象基类（Abstract Base Classes, ABCs）。 
    #抽象基类是一种特殊的类，它不能被直接实例化，而是用来定义其他类应该具有的共同接口。
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def get_latest_bar(self, symbol):
        """
        Returns the last bar updated.
        """
        raise NotImplementedError("Should implement get_latest_bar()")
        
    @abstractmethod
    def get_latest_bars(self, symbol, N=1):
        """
        Returns the last N bars updated.
        """
        raise NotImplementedError("Should implement get_latest_bars()")
        
    @abstractmethod
    def get_latest_bar_datetime(self, symbol):
        """
        Returns a Python datetime object for the last bar. """
        raise NotImplementedError("Should implement get_latest_bar_datetime()")
        
    @abstractmethod
    def get_latest_bar_value(self, symbol, val_type): 
        """
        Returns one of the Open, High, Low, Close, Volume or OI from the last bar.
        """
        raise NotImplementedError("Should implement get_latest_bar_value()")
        
    @abstractmethod
    def get_latest_bars_values(self, symbol, val_type, N=1):
        """
        Returns the last N bar values from the
        latest_symbol list, or N-k if less available. """
        raise NotImplementedError("Should implement get_latest_bars_values()")
        
    @abstractmethod
    def update_bars(self):
        """
         Pushes the latest bars to the bars_queue for each symbol
        in a tuple OHLCV format: (datetime, open, high, low,
        close, volume).
        """
   
        raise NotImplementedError("Should implement update_bars()")



# 继承抽象基类
# 
class HistoricFromAkshareDataHandler(DataHandler):
    """
    HistoricCSVDataHandler is designed to read CSV files 
    """
    def __init__(self, events, market, symbol_list):
        self.events = events
        self.market = market
        self.symbol_list = symbol_list
        self.symbol_data = {}
        self.latest_symbol_data = {}
        self.continue_backtest = True
        self._get_akshare_data()
        
        
    def _get_akshare_data(self): 
        """
        Get data from akshare, converting
        them into pandas DataFrames within a symbol dictionary.
        """
        comb_index = None
        for s in self.symbol_list:
            self.symbol_data[s] = ak.futures_zh_daily_sina(s)           
            #print("----")
            #print(self.symbol_data[s])
            self.latest_symbol_data[s] = pd.DataFrame()
        # Reindex the dataframes
       
        for s in self.symbol_list:
            comb_index = self.symbol_data[s]['date']
            self.symbol_data[s] = self.symbol_data[s].set_index(comb_index)
            #print(self.symbol_data[s])

        
    def _get_new_bar(self, symbol): 
        """
        Returns the latest bar from the data feed. 
        """
        if len(self.symbol_data[symbol]) > 0:
            return self.symbol_data[symbol]
        return None
        
    def get_latest_bar(self, symbol): 
        """
            Returns the last bar from the latest_symbol list.
        """
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data set.") 
            raise
        else:
            #print(type(bars_list))
            #print(bars_list)
            return bars_list[-1:]
    
    def get_latest_bars(self, symbol, N=1): 
        """
        Returns the last N bars from the latest_symbol list,
        or N-k if less available. 
        """
        try:
            bars_list = self.latest_symbol_data[symbol] 
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise 
        else:
            return bars_list[-N:]

    def get_latest_bar_datetime(self, symbol): 
        """
        Returns a Python datetime object for the last bar. 
        """
        try:
            bars_list = self.latest_symbol_data[symbol].index
        except KeyError:
            print("That symbol is not available in the historical data set.") 
            raise
        else:
            return bars_list[-1]
        
    def get_latest_bar_value(self, symbol, val_type): 
        """
        Returns one of the Open, High, Low, Close, Volume or OI
        values from the pandas Bar series object.
        """
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise 
        else:
            return (bars_list.iloc[-1][val_type])
    
    def get_latest_bars_values(self, symbol, val_type, N=1): 
            """
            Returns the last N bar values from the latest_symbol list, or N-k if less available. 
            """
            try:
                bars_list = self.latest_symbol_data[symbol] 
            except KeyError:
                print("That symbol is not available in the historical data set.")
                raise 
            else:
                return bars_list.iloc[-N:][val_type].to_numpy()

    def update_bars(self): 
        """
            Pushes the latest bar to the latest_symbol_data structure for all symbols in the symbol list.
        """
        for s in self.symbol_list: 
            try:
                bar = self._get_new_bar(s)
            except StopIteration:
                self.continue_backtest = False 
            else:
                if bar is not None: 
                    # 添加新增行情到行情数据尾部
                    self.latest_symbol_data[s] = pd.concat([self.latest_symbol_data[s], bar], axis=0)
        # 发送行情通知事件
        #self.events.put(MarketEvent())
        

        
hisData = HistoricFromAkshareDataHandler(None, "", ["V2403", "P2403", "B2403", "M2403"])
hisData.update_bars()
print("获取最新的一条数据--->") 
print(hisData.get_latest_bar("V2403"))
print("获取最新的3条数据--->") 
print(hisData.get_latest_bars("V2403", 3))
print("获取最新日期--->") 
print(hisData.get_latest_bar_datetime("V2403"))  # out : 2024-2-23

print("获取最新的收盘价--->") 
print(hisData.get_latest_bar_value("V2403", "close")) # out : 290.8
 
print("获取最新的收盘价集合--->") 
close_np = hisData.get_latest_bars_values("V2403", "close" , 3) # out : [288.8 291.4 290.8]
print(type(close_np))
print(close_np)
#print(hisData.get_latest_bar("0700"))

获取最新的一条数据--->
                  date    open    high     low   close  volume  hold  settle
date                                                                        
2024-03-14  2024-03-14  5520.0  5537.0  5503.0  5532.0     701     0  5561.0
获取最新的3条数据--->
                  date    open    high     low   close  volume   hold  settle
date                                                                         
2024-03-12  2024-03-12  5569.0  5569.0  5521.0  5535.0     589  25402  5540.0
2024-03-13  2024-03-13  5593.0  5614.0  5520.0  5520.0    1279  25161  5566.0
2024-03-14  2024-03-14  5520.0  5537.0  5503.0  5532.0     701      0  5561.0
获取最新日期--->
2024-03-14
获取最新的收盘价--->
5532.0
获取最新的收盘价集合--->
<class 'numpy.ndarray'>
[5535. 5520. 5532.]


##### 4 模拟从akshare 实时获取TICK

In [127]:
# 继承抽象基类
# 
class TickAKShareDataHandler(DataHandler):
    """
    HistoricCSVDataHandler is designed to read CSV files 
    """
    def __init__(self, events, market, symbol_list):
        self.events = events
        self.market = market
        self.symbol_list = symbol_list
        self.symbol_data = {}
        self.latest_symbol_data = {}
        self.continue_backtest = True
        self._get_akshare_data()
        # 事件处理线程
        self._task_thread = threading.Thread(target = self._run)
        # 模拟tick 3秒请求一次
        self._timerSleep= 3 
        self._stop = True
        
        

    def _run(self):
        
        while not self._stop :
            time.sleep(self.timerSleep_)
            _get_akshare_data(self)
            update_bars(self)
        
    def _get_akshare_data(self): 
        """
        Get data from akshare, converting
        them into pandas DataFrames within a symbol dictionary.
        """
        comb_index = None
        for s in self.symbol_list:
            self.symbol_data[s] = ak.futures_zh_spot(s)           
            #print("----")
            #print(self.symbol_data[s])
            self.latest_symbol_data[s] = pd.DataFrame()
        # Reindex the dataframes
       
        for s in self.symbol_list:
            comb_index = self.symbol_data[s]['time']
            self.symbol_data[s] = self.symbol_data[s].set_index(comb_index)
            #print(self.symbol_data[s])

        
    def _get_new_bar(self, symbol): 
        """
        Returns the latest bar from the data feed. 
        """
        if len(self.symbol_data[symbol]) > 0:
            return self.symbol_data[symbol]
        return None
    
    def start(self):
        self._stop = False
        self._task_thread.start()
        
    def get_latest_bar(self, symbol): 
        """
            Returns the last bar from the latest_symbol list.
        """
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data set.") 
            raise
        else:
            #print(type(bars_list))
            #print(bars_list)
            return bars_list[-1:]
    
    def get_latest_bars(self, symbol, N=1): 
        """
        Returns the last N bars from the latest_symbol list,
        or N-k if less available. 
        """
        try:
            bars_list = self.latest_symbol_data[symbol] 
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise 
        else:
            return bars_list[-N:]

    def get_latest_bar_datetime(self, symbol): 
        """
        Returns a Python datetime object for the last bar. 
        """
        try:
            bars_list = self.latest_symbol_data[symbol].index
        except KeyError:
            print("That symbol is not available in the historical data set.") 
            raise
        else:
            return bars_list[-1]
        
    def get_latest_bar_value(self, symbol, val_type): 
        """
        Returns one of the Open, High, Low, Close, Volume or OI
        values from the pandas Bar series object.
        """
        try:
            bars_list = self.latest_symbol_data[symbol]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise 
        else:
            return (bars_list.iloc[-1][val_type])
    
    def get_latest_bars_values(self, symbol, val_type, N=1): 
            """
            Returns the last N bar values from the latest_symbol list, or N-k if less available. 
            """
            try:
                bars_list = self.latest_symbol_data[symbol] 
            except KeyError:
                print("That symbol is not available in the historical data set.")
                raise 
            else:
                return bars_list.iloc[-N:][val_type].to_numpy()

    def update_bars(self): 
        """
            Pushes the latest bar to the latest_symbol_data structure for all symbols in the symbol list.
        """
        for s in self.symbol_list: 
            try:
                bar = self._get_new_bar(s)
            except StopIteration:
                self.continue_backtest = False 
            else:
                if bar is not None: 
                    # 添加新增行情到行情数据尾部
                    self.latest_symbol_data[s] = pd.concat([self.latest_symbol_data[s], bar], axis=0)
        # 发送行情通知事件
        #self.events.put(MarketEvent())
        

        
hisData = TickAKShareDataHandler(None, "", ["V2403", "P2403", "B2403", "M2403"])
hisData.update_bars()
print("获取最新的一条数据--->") 
print(hisData.get_latest_bar("V2403"))
print("获取最新的3条数据--->") 
print(hisData.get_latest_bars("V2403", 3))
print("获取最新日期--->") 
print(hisData.get_latest_bar_datetime("V2403"))  # out : 2024-2-23

print("获取最新的收盘价--->") 
print(hisData.get_latest_bar_value("V2403", "last_close")) # out : 290.8
 
print("获取最新的收盘价集合--->") 
close_np = hisData.get_latest_bars_values("V2403", "last_close" , 3) # out : [288.8 291.4 290.8]
print(type(close_np))
print(close_np)
#print(hisData.get_latest_bar("0700"))

获取最新的一条数据--->
         symbol    time    open    high     low  current_price  bid_price  \
time                                                                        
150230  PVC2403  150230  5569.0  5606.0  5541.0         5600.0     5582.0   

        ask_price  buy_vol  sell_vol     hold  volume  avg_price  last_close  \
time                                                                           
150230     5606.0       10         2  42171.0    5094     5580.0      5600.0   

        last_settle_price  
time                       
150230             5565.0  
获取最新的3条数据--->
         symbol    time    open    high     low  current_price  bid_price  \
time                                                                        
150230  PVC2403  150230  5569.0  5606.0  5541.0         5600.0     5582.0   

        ask_price  buy_vol  sell_vol     hold  volume  avg_price  last_close  \
time                                                                           
150230     5606.0      